In [1]:
#start


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1727667530908_0013,pyspark,idle,Link,Link,user_n-renukumar_rajendran_draup_com,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.addPyFile("s3://jobs-emr-etl-config/jobs_run/prod/cc/ML_ETL_Config.py")
from ML_ETL_Config import *
utils_module = MlEtlUtils(spark, sc)
to_import = [name for name in dir(utils_module) if not (name.startswith('_') | (name=="spark") | (name=="sc"))]
globals().update({name: getattr(utils_module, name) for name in to_import})
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

404
404
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gateway.draup.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(

In [3]:
linkedin=spark.read.format('delta').load('s3://etl-datahouse/prod/dcd/xverum/delta/').selectExpr('account_name as linkedin_Name','social_url as Linkedin_link','website as Linkedin_website','permalink','industry','headquarter','about','company_size','employee_count','year_established','logo_url','account_type','date_updated')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
linkedin.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['linkedin_Name', 'Linkedin_link', 'Linkedin_website', 'permalink', 'industry', 'headquarter', 'about', 'company_size', 'employee_count', 'year_established', 'logo_url', 'account_type', 'date_updated']

# using domain

In [5]:
df=spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/LinkedIn_permalink_27-09-24.csv",header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Name', 'Cleaned Url', 'Linkedin Link']

In [7]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1048

In [8]:
#cleaning domain urls
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a UDF to extract the domain name from a URL https://www.linkedin.com/company/draupplatform/
def extract_parma(url):
    try:
        if "www." in url:
            url = url.split("www.")[1].split('/')[0]
        elif "//"in url:
            url = url.split("//")[1].split('/')[0]
        return url
    except:
        return url

# Register the UDF with Spark
extract_parma_udf = udf(extract_parma, StringType())

df1 = df.withColumn('domain_url', extract_parma_udf(df['Linkedin Link'])).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df1.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Name', 'Cleaned Url', 'Linkedin Link', 'domain_url']

# Cleaning linkedin domain

In [10]:
#cleaning domain urls
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a UDF to extract the domain name from a URL https://www.linkedin.com/company/draupplatform/
def extract_parma(url):
    try:
        if "www." in url:
            url = url.split("www.")[1].split('/')[0]
        elif "//"in url:
            url = url.split("//")[1].split('/')[0]
        return url
    except:
        return url

# Register the UDF with Spark
extract_parma_udf = udf(extract_parma, StringType())

cmp1 = linkedin.withColumn('Linkedin_domain', extract_parma_udf(linkedin['Linkedin_website']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
cmp1.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['linkedin_Name', 'Linkedin_link', 'Linkedin_website', 'permalink', 'industry', 'headquarter', 'about', 'company_size', 'employee_count', 'year_established', 'logo_url', 'account_type', 'date_updated', 'Linkedin_domain']

In [12]:
main_df=df1.join(cmp1,df1['domain_url']==cmp1['Linkedin_domain'],how="left")#.distinct().drop("Linkedin_domain").filter(col('Domain URL')!="linkedin.com")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
main_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Name', 'Cleaned Url', 'Linkedin Link', 'domain_url', 'linkedin_Name', 'Linkedin_link', 'Linkedin_website', 'permalink', 'industry', 'headquarter', 'about', 'company_size', 'employee_count', 'year_established', 'logo_url', 'account_type', 'date_updated', 'Linkedin_domain']

In [14]:
main_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20508860

In [15]:
main_df.groupBy("domain_url").count().orderBy('count',ascending=False).show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+
|domain_url     |count   |
+---------------+--------+
|linkedin.com   |20507634|
|in.linkedin.com|648     |
|NULL           |351     |
|uk.linkedin.com|172     |
|es.linkedin.com|54      |
|gt.linkedin.com|1       |
+---------------+--------+

In [16]:
main_df1 = main_df.groupBy("domain_url").count().orderBy('count',ascending=False).filter(F.col("count") <=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df1 = df1.withColumnRenamed("domain_url", "domain_url1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
fin_df = df1.join(main_df1,df1['domain_url1']==main_df1['domain_url'],how="left").drop('domain_url1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
main_df1 = fin_df.join(cmp1,fin_df['domain_url']==cmp1['Linkedin_domain'],how="left").drop('domain_url1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
main_df1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1048

In [22]:
main_df1.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Company Name', 'Cleaned Company Url', 'domain_url', 'count', 'linkedin_Name', 'Linkedin_link', 'Linkedin_website', 'permalink', 'industry', 'headquarter', 'about', 'company_size', 'employee_count', 'year_established', 'logo_url', 'account_type', 'date_updated', 'Linkedin_domain']

In [ ]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=main_df1.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Linkedin_Details_use_domain_26-09-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# using name

In [30]:
df=spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/LinkedIn_permalink_27-09-24.csv",header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Name', 'Cleaned Url', 'Linkedin Link']

In [32]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1048

In [33]:
df1=df.withColumn('lower_name',lower(col('Name')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
linkedin1=linkedin.withColumn('lower_name_',lower(col('linkedin_Name')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
fin_df=df1.join(linkedin1,df1["lower_name"]==linkedin1["lower_name_"],how="left").drop("lower_name_","lower_name")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
fin_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1425

In [37]:
fin_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Name', 'Cleaned Url', 'Linkedin Link', 'linkedin_Name', 'Linkedin_link', 'Linkedin_website', 'permalink', 'industry', 'headquarter', 'about', 'company_size', 'employee_count', 'year_established', 'logo_url', 'account_type', 'date_updated']

In [58]:
#fin_df1 = fin_df.select('LinkedIn Name','employee_count')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=fin_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Linkedin_Details_use_name_27-09_2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '7JF5EMA0H19HDMEK', 'HostId': '2A/leN4Vkb1XMalmTWK1I93vQAlmbqETQof7TGd7N9CoVtL2G0XwW+HMwnivk0fatttFPt5nXQ0pkRj3KrnRjA==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '2A/leN4Vkb1XMalmTWK1I93vQAlmbqETQof7TGd7N9CoVtL2G0XwW+HMwnivk0fatttFPt5nXQ0pkRj3KrnRjA==', 'x-amz-request-id': '7JF5EMA0H19HDMEK', 'date': 'Fri, 27 Sep 2024 08:57:13 GMT', 'x-amz-expiration': 'expiry-date="Thu, 03 Oct 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"4f28ff1f72a19381cc2c6ee377935556"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 03 Oct 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"4f28ff1f72a19381cc2c6ee377935556"', 'ServerSideEncryption': 'AES256'}

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re


def generalize_company_name(name):
    name = str(name).lower()
    name = re.sub(r'\([^)]*\)', '', name)
    # end_char = r'[^\w\s]+$'
    name = re.sub(r'[^\w\s]+$', '', name)
    # ms_pattern = r'^m/s[^\w\s]*\s*'
    name = re.sub(r'^m/s[^\w\s]*\s*', '', name)
    name = re.sub(' +', ' ', name)
    name = name.strip()
    common_terms = [
        'PRIVATE.', 'PVT.', 'PVT. LTD.', 'PVT. LTD', 'CO.', 'CORPORATION',
        'CORP', 'INC.', 'INC', 'LLC', 'LLLP.', 'LIMITED LIABILITY', 'L.L.C.',
        'L.L.P.', 'L.P.', 'L.L.L.P.', 'L.L.L.P', 'LTD.', 'LIMITED.',
        'PRIVATE LIMITED.', 'PVT LTD.', 'PVT. LTD.', 'PVT. LTD', 'INDIA.',
        'PRIVATE', 'PVT', 'LTD', 'LIMITED', 'PRIVATE LIMITED', 'PVT LTD', 'CO',
        'LLP', 'LP', 'LLLP', '&', '\.', '\(\)', '(P)', 'Corporates', 'Corporate'
        'SOLUTIONS', 'SOLUTION', 'P.', 'P', '\.', 'india.', 'india',
        'Liabili Ty', 'Servic Es', 'Limi Ted', 'Private.Limited.', 'Limted',
        'Lim Ited', 'Lmited', 'Limted', 'L Imited', 'Liability', 'L iability',
        'Li ability', 'Lia bility', 'Liab ility', 'Liabi lity', 'Liabil ity',
        'Liabilit y', 'L imited', 'Lim ited', 'Limit ed', 'Limite d',
        'Services', 'Service', 'PARTNERSHIPS', 'PARTNERSHIP', 'TECHNOLOGIES',
        'GROUP', 'TECHNOLOGIE', 'TECHNOLOGY', 'Infotech'
    ]
    for term in common_terms:
        og_name = name
        name = re.sub(rf'(?i)\s{term}$', '', name, flags=re.IGNORECASE).strip()
        if og_name != name:
            name = generalize_company_name(name)

    name = re.sub(' +', ' ', name).replace(" ","")
    return name


# Register the UDF with Spark
clean_name_udf = udf(generalize_company_name, StringType())

cmp = linkedin1.withColumn('cleaned_name', clean_name_udf(linkedin1['lower_name_'])).drop('clean_name')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
join_df=df1.join(cmp,df1["lower_name"]==cmp["cleaned_name"],how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
join_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [27]:
df1.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Account Name', 'lower_name']

In [14]:
join_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Account Name', 'lower_name', 'linkedin_Name', 'Linkedin_link', 'Linkedin_website', 'permalink', 'industry', 'headquarter', 'about', 'company_size', 'employee_count', 'year_established', 'logo_url', 'account_type', 'date_updated', 'lower_name_', 'cleaned_name']

In [15]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=join_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Linkedin_Details_using_clean_name_30-05_2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '0PHJAPEE50DXHS1T', 'HostId': 'sPBPLo1n9gIJxIO0Cb9OvzkbWq5qVOti0vnQYwzC7kcwkkacqMdc6i5gJMi02Yd3/9f6RGQf4q4=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'sPBPLo1n9gIJxIO0Cb9OvzkbWq5qVOti0vnQYwzC7kcwkkacqMdc6i5gJMi02Yd3/9f6RGQf4q4=', 'x-amz-request-id': '0PHJAPEE50DXHS1T', 'date': 'Thu, 30 May 2024 08:12:57 GMT', 'x-amz-expiration': 'expiry-date="Wed, 05 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"3ac879349381f5d78984e280f5a4c7f3"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Wed, 05 Jun 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"3ac879349381f5d78984e280f5a4c7f3"', 'ServerSideEncryption': 'AES256'}

# using permalink

In [5]:
df=spark.read.csv("s3://draup-command-center/temp/cc/Renukumar_Draup/karthika/Input/linkedin_permalink_input_30-09-2024.csv",header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Name', 'Linkedin Url']

In [7]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2690

In [8]:
# creating permalink
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a UDF to extract the domain name from a URL https://www.linkedin.com/company/draupplatform/
def extract_parma(url):
    try:
        if "/company/" in url:
            url = url.split("/company/")[1].split('/')[0]
        elif "/cmp/"in url:
            url = url.split("/cmp/")[1].split('/')[0]
        elif "/school/" in url:
            url = url.split("/school/")[1].split('/')[0]
        elif "/showcase/" in url:
            url = url.split("/showcase/")[1].split('/')[0]
        elif "/company-beta/" in url:
            url = url.split("/company-beta/")[1].split('/')[0]
        elif "/in/" in url:
            url = url.split("/in/")[1].split('/')[0]
        return url
    except:
        return url

# Register the UDF with Spark
extract_parma_udf = udf(extract_parma, StringType())

df1 = df.withColumn('linkedin_perma', extract_parma_udf(df['Linkedin Url'])).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df1.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Name', 'Linkedin Url', 'linkedin_perma']

In [10]:
fin_df=df1.join(linkedin,df1["linkedin_perma"]==linkedin["permalink"],how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
fin_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Name', 'Linkedin Url', 'linkedin_perma', 'linkedin_Name', 'Linkedin_link', 'Linkedin_website', 'permalink', 'industry', 'headquarter', 'about', 'company_size', 'employee_count', 'year_established', 'logo_url', 'account_type', 'date_updated']

In [12]:
fin_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2690

In [34]:
#fin_df1 = fin_df[['Company Name', 'Linkedin Url','linkedin_perma','linkedin_Name','Linkedin_website','year_established','headquarter','industry']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=fin_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Linkedin_details_use_permalink_30-09-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '4A1CGP8RR9GT4B95', 'HostId': 'x1H+B5WTGoppU8bEVgZJC35ggRUrE/oflPWiZfamSXnFlm47HzujXJiYd4a+w/Kt82DbjfYBA/h6IfAPZzu5wA==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'x1H+B5WTGoppU8bEVgZJC35ggRUrE/oflPWiZfamSXnFlm47HzujXJiYd4a+w/Kt82DbjfYBA/h6IfAPZzu5wA==', 'x-amz-request-id': '4A1CGP8RR9GT4B95', 'date': 'Mon, 30 Sep 2024 05:35:16 GMT', 'x-amz-expiration': 'expiry-date="Sun, 06 Oct 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"982c728fd4e0e725904126029c874c72"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Sun, 06 Oct 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"982c728fd4e0e725904126029c874c72"', 'ServerSideEncryption': 'AES256'}

In [ ]:
linkedin.printSchema()

In [15]:
exp_df=linkedin.select('*', explode(col("industry")).alias('linkedin_industry'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
fin_df=exp_df.filter(exp_df.linkedin_industry =='Government Administration')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
linkedin.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

49108470

In [15]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=fin_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/Avinash/Output/Linkedin_Government Administration_13-05-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'GAQ8R1QAADJZQ3Y0', 'HostId': 'AaRnMWNuJzrFBPmz/G8vqAbaCMJa9ksWV+q3jnVQ4Edr9LyOIsBRCct3D+ryadSKFcvKsGR8t+oC3q0x5croOg==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'AaRnMWNuJzrFBPmz/G8vqAbaCMJa9ksWV+q3jnVQ4Edr9LyOIsBRCct3D+ryadSKFcvKsGR8t+oC3q0x5croOg==', 'x-amz-request-id': 'GAQ8R1QAADJZQ3Y0', 'date': 'Mon, 13 May 2024 10:25:23 GMT', 'x-amz-expiration': 'expiry-date="Sun, 19 May 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"cbaa722d56c0533b0979cf8c30a58623"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Sun, 19 May 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"cbaa722d56c0533b0979cf8c30a58623"', 'ServerSideEncryption': 'AES256'}

In [36]:
linkedin.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- linkedin_Name: string (nullable = true)
 |-- Linkedin_link: string (nullable = true)
 |-- Linkedin_website: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- industry: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- headquarter: string (nullable = true)
 |-- about: string (nullable = true)
 |-- company_size: string (nullable = true)
 |-- employee_count: long (nullable = true)
 |-- year_established: string (nullable = true)
 |-- logo_url: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- date_updated: string (nullable = true)

In [42]:
group_df = linkedin.groupBy('account_type').count().orderBy(desc('count'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
group_df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['account_type', 'count']

In [44]:
group_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

147

In [45]:
from io import StringIO # python3; python2: BytesIO import boto3
import boto3


bucket = 'draup-command-center' # already created on S3
csv_buffer = StringIO()

res2=group_df.toPandas()
res2.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'temp/cc/Renukumar_Draup/karthika/output/Linkedin_account_count_09-08-2024.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '1ZMWDHM5PCWD1QBM', 'HostId': 'zniNA+A/ImujRfC63ehLyK2I51IuxEG7IAb0/uJbxq9YwTRkmGiDsfnxZ8zrekTxwIRr1RlIX5IMMRBgFPgTzw==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'zniNA+A/ImujRfC63ehLyK2I51IuxEG7IAb0/uJbxq9YwTRkmGiDsfnxZ8zrekTxwIRr1RlIX5IMMRBgFPgTzw==', 'x-amz-request-id': '1ZMWDHM5PCWD1QBM', 'date': 'Fri, 09 Aug 2024 06:33:45 GMT', 'x-amz-expiration': 'expiry-date="Thu, 15 Aug 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'x-amz-server-side-encryption': 'AES256', 'etag': '"cb95b861710272196f4852a5e093b4d2"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 1}, 'Expiration': 'expiry-date="Thu, 15 Aug 2024 00:00:00 GMT", rule-id="Delete in temp folder"', 'ETag': '"cb95b861710272196f4852a5e093b4d2"', 'ServerSideEncryption': 'AES256'}